In [1]:
import os
import re
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sk

In [2]:
pip install webcolors

Note: you may need to restart the kernel to use updated packages.


In [3]:
import webcolors

In [4]:
ess9 = pd.read_csv(os.path.relpath("Data/ESS9-20220626.csv"))
print(ess9.shape)
#ess9.iloc[:, [5,9]]

(49519, 32)


In [5]:
#https://www.eea.europa.eu/data-and-maps/data/waterbase-lakes-4/country-codes-and-abbreviations-32-records/country-codes-and-abbreviations-32-records
key = ess9.iloc[:, [5,9]]
cntrycodes = pd.read_csv(os.path.relpath("Data/iso3166_alpha2_codes.csv"))
countrykey = pd.merge(key, cntrycodes, left_on = "cntry", right_on = "ISO2")
novcolor = pd.read_csv(os.path.relpath("Data/ColorTablePCmap-20220627.txt"), sep='\t', header = None)
novcolor.columns = ['country', 'color']
novcolor.loc[16,'color'] = 'red1'
countrycolors = pd.merge(countrykey, novcolor, how = 'left', left_on = 'Country name', right_on = 'country')
countrycolors = countrycolors[['region', 'ISO2', 'country', 'color']]

In [6]:
# remove nonanswer answers
#ess9.columns.get_loc("ipcrtiv"), ess9.columns.get_loc("impfun")
ess9 = ess9.set_index('region').loc[:, 'ipcrtiv':'impfun']
ess9.replace(7, np.NaN, inplace = True)
ess9.replace(8, np.NaN, inplace = True)
ess9.replace(9, np.NaN, inplace = True)

In [7]:
# mean value per nut. reg means nutregion
ess9reg = ess9.groupby('region').mean()
ess9reg.head()
# ppca for nan, https://stackoverflow.com/questions/29420737/pca-with-missing-values-in-python
# https://ess-search.nsd.no/en/study/bdc7c350-1029-4cb3-9d5e-53f668b8fa74

,ipcrtiv,imprich,ipeqopt,ipshabt,impsafe,impdiff,ipfrule,ipudrst,ipmodst,ipgdtim,...,iphlppl,ipsuces,ipstrgv,ipadvnt,ipbhprp,iprspot,iplylfr,impenv,imptrad,impfun
region,,,,,,,,,,,,,,,,,,,,,
99999,2.000000,6.000000,1.000000,3.000000,4.000000,2.000000,5.000000,2.000000,3.000000,2.000000,...,2.000000,2.000000,5.000000,5.000000,5.000000,5.000000,2.000000,1.000000,5.000000,3.000000
AT11,2.726190,4.423529,1.940476,2.602410,2.000000,3.642857,2.951807,2.494118,2.811765,2.717647,...,1.951807,2.929412,2.130952,4.258824,2.082353,2.294118,1.529412,1.588235,1.870588,2.917647
AT12,2.708333,4.111842,2.386813,2.671053,2.103070,3.288840,2.869757,2.521930,3.026374,2.798246,...,2.245077,3.037118,2.039911,4.048035,2.324503,2.447368,1.634573,1.772926,2.531729,2.888158
AT13,2.258252,3.904669,1.867446,2.736842,2.223301,3.003899,3.159844,2.127202,2.994163,2.526316,...,2.040777,2.753398,2.365759,3.923977,2.430799,2.727984,1.581712,1.813592,3.001946,2.935922
AT21,2.558282,4.104294,2.006135,2.625767,1.662577,3.300613,3.155280,2.257669,2.705521,2.546584,...,2.024540,2.901235,2.000000,4.202454,2.171779,2.533742,1.693252,1.883436,2.147239,2.732919


In [8]:
nuts = gpd.read_file(os.path.relpath('Data/NUTS_RG_20M_2021_4326-20220626.geojson'))

SyntaxError: unexpected EOF while parsing (3183756239.py, line 1)

In [ ]:
gdf_ess9 = pd.merge(nuts, ess9reg, left_on = 'NUTS_ID', right_index = True)

In [ ]:
#Check it looks right
fig, ax = plt.subplots(figsize=(12, 10))
#euro.plot(ax=ax, )
gdf_ess9.plot('impdiff', ax=ax)
gdf_nodes.plot(ax=ax, color = 'green') #london
ax.axis('off')
ax.legend()
plt.axis('equal')
plt.show()

In [ ]:
place = 'City of London, Greater London, England, United Kingdom'
G = ox.graph_from_place(place, network_type='drive', simplify=False)
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
#fig, ax = ox.plot_graph(G)

In [ ]:
ess9nodes = gpd.sjoin(gdf_nodes, gdf_ess9)

In [ ]:
jumps = [25, 26, 32, 33]

for j in jumps:
    level = jumps.index(j)
    # retrieve the nodes in this jump
    jump = pd.read_csv(os.path.relpath("run_spercolation_outputs/membTables/membership_p"+str(j)+'.txt')) 
    jump['level'] = level
    jump = jump[['level', 'node_id', 'cluster_id']]
    
    # subset clusters of a minimum size
    sizes = pd.read_csv(os.path.relpath("run_spercolation_outputs/cluster_sizes/clust_size_p"+str(j)+'.txt'))
    clusters = sizes[sizes.cluster_size > 2] 
    nodes = pd.merge(jump, clusters, on='cluster_id') #.add_suffix(level) 

    # add the ess9 weights to the nodes in this jump
    ess9jumpnodes = pd.merge(ess9nodes, nodes, how = 'right', left_index = True, right_on = 'node_id')
    
    # find the weighted average ess9 value for each cluster in this jump
    ess9jumpclusters = ess9jumpnodes.groupby('cluster_id').mean().sort_values('cluster_size', ascending = False)
    
    # calculate similarity between two largest clusters
    largest = ess9jumpclusters.iloc[[0]].loc[:,'ipcrtiv':'impfun']
    larger = ess9jumpclusters.iloc[[1]].loc[:,'ipcrtiv':'impfun']
    print(sk.pairwise.cosine_similarity(largest, larger))

# PCA

In [ ]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=21, whiten=True) 
pca.fit(ess9reg)
explained_variance = pca.explained_variance_ratio_
singular_values = pca.singular_values_

In [ ]:
x = np.arange(1,len(explained_variance)+1)
plt.plot(x, explained_variance)
plt.ylabel('Share of Variance Explained')
plt.xticks(np.arange(min(x), max(x)+1, 1.0))
plt.show()

In [ ]:
for i in range(0, 20):
    print(f"Component {i:>2} accounts for {explained_variance[i]*100:>2.2f}% of variance")

In [ ]:
keep_n_components = 14

# If we weren't changing the number of components we
# could re-use the pca object created above. 
pca = PCA(n_components=keep_n_components, whiten=True)

X_train = pca.fit_transform(ess9reg)

# Notice that we get the _same_ values out,
# so this is a *deterministic* process that
# is fully replicable (allowing for algorithmic
# and programming language differences).
for i in range(0, keep_n_components):
    print(f"Component {i:>2} accounts for {pca.explained_variance_ratio_[i]*100:>2.2f}% of variance")

# Notice...
print(len(X_train))
print(ess9reg.shape[0])
# So each observation has a row in X_train and there is 
# 1 column for each component. This defines the mapping
# of the original data space into the reduced one
print(len(X_train[0])) 

In [ ]:
for x in [X_train]:
    new_columns = []
    
    for i in range(0,keep_n_components):
        new_columns.append([])

    for i in x:
        for j in range(0,keep_n_components):
            new_columns[j].append(i[j])

    for i in range(0,keep_n_components):
        ess9reg[f"Component {i+1}"] = new_columns[i]

In [ ]:
ess9cntry = pd.merge(ess9reg, countrycolors, left_index = True, right_on = "region")

In [ ]:
ess9cntry

In [ ]:
ess9cntry.to_csv(os.path.relpath("Data/ess9cntry.csv"))

In [ ]:
#euro = gpd.read_file(os.path.relpath("Data/ref-countries-2020-10m.geojson/CNTR_BN_10M_2020_3035_INLAND.geojson"))